### To get API key: https://developer.nytimes.com/get-started

In [1]:
from pynytimes import NYTAPI
import datetime
import pandas as pd

import numpy as np

In [ ]:
def get_news_specific_stock(start_date: datetime.datetime, end_date: datetime.datetime, nyt=None, k=100):
    """
    get top k most relevent finance news headings on each day from NY times
    """
    res = []
    try:
        #if not nyt:
        #    nyt = NYTAPI(api_key, parse_dates=True)

        articles = nyt.article_search(
            query = "Microsoft", 
            results = k,
            dates = {
                "begin": start_date,
                "end": end_date
            },
            options = {
                "sort": "relevance", 
                "sources": [
                    "New York Times",
                    "AP",
                    "Reuters",
                    "International Herald Tribune"
                ],
                # Only get information from these news desk
                "news_desk": [
                    "Business",
                    "Financial",
                    "Technology"
                ],
                "body": [
                    "Microsoft",
                ],
            }
        )
        
        for i in range(len(articles)):
            timestamp = articles[i]['pub_date']
            article_url = articles[i]['web_url']
            lead_paragraph = articles[i]['lead_paragraph']
            abstract = articles[i]['abstract']
            #print(articles[i])
            res.append({'timestamp': timestamp,
                        'article_url': article_url,
                        'lead_paragraph': lead_paragraph,
                        'abstract': abstract,
                        })
        
    except Exception as e:
        res.append({})
    
    df = pd.DataFrame(data=res)

    return df

### Change api key, start and end date

In [ ]:
api_key = "uLZTQY03vZfRZfk6nmzKcSc82EEUj4lK"

start_date = datetime.datetime(2024, 1, 1)
end_date = datetime.datetime(2024, 1, 31)

### Get articles

In [ ]:
nyt = NYTAPI(api_key, parse_dates=True)
k_results = (end_date - start_date).days * 1.2  # How many results to pull for. (x1.2 as an estimate for the max articles that exist.)

article_df = get_news_specific_stock(start_date, end_date, nyt, k=k_results)

In [ ]:
article_df

#### Check if the news falls before or after market closes on each day. If it's after close, then the news should be categorised as a next day news.

In [ ]:
threshold_time = pd.Timestamp('16:00:00').time()

article_df['adjusted_date'] = article_df['timestamp'].apply(lambda x: x.date() if x.time() < threshold_time else (x + pd.Timedelta(days=1)).date())
article_df.head()

### Save file 

In [ ]:
filename = "../data/nyt_microsoft.csv"
article_df.to_csv(filename, index=False)